In [1]:
#!/usr/bin/python
#-*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import re
import time
from multiprocessing import Pool
import multiprocessing
import os


In [2]:
start_time = time.time()
print ("start time : " + str(start_time) )

fileList = ["TB_ADM_TV_API_LOG_02_2020-02-05"]
FOLDER_PATH = "/home/yeongbin/code_folder/KIC/"
numCores =  multiprocessing.cpu_count()
print('cpu counts : %d' % numCores)

globalDF = pd.read_csv(FOLDER_PATH+fileList[0]+".txt", delimiter='\t', encoding='UTF8')
print(globalDF.index)

# for i in range(1,10):
#     tempDF = pd.read_csv(FOLDER_PATH+fileList[i]+".txt", delimiter='\t', encoding='UTF8')
#     globalDF = globalDF.append(tempDF, ignore_index=True)
#     print(globalDF.index)
    
    
#tempDF = pd.read_csv(FOLDER_PATH+fileList[1]+".txt", delimiter='\t', encoding='UTF8')
#print(tempDF.size)
#temp1DF = pd.read_csv(FOLDER_PATH+fileList[2]+".txt", delimiter='\t', encoding='UTF8')
#print(temp1DF.size)

#
#append 할 때 index가 같은 것을 append하면 Series로 합쳐져서 lower()함수를 사용했을 때 에러가 나는 문제가 있다.
#그레서 append 하는 메서드에 파라메터로 ignore_index=True 옵션을 주어서 인덱스 무시하고 아래부분에 이어붙일 수 있게 하였다.
#
#globalDF = globalDF.append(tempDF, ignore_index=True)
#globalDF = globalDF.append(temp1DF, ignore_index=True)

###테스트용
###globalDF = pd.read_excel(FOLDER_PATH+"sample_20000.xlsx", encoding='UTF8')

#3만 인덱스까지 데이터 슬라이싱 하는 부분
#globalDF = globalDF[0:30000]


print ("소요 시간(초) : " + str(time.time() - start_time))

start time : 1584510481.9203036
cpu counts : 16


/home/yeongbin/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


RangeIndex(start=0, stop=1143298, step=1)
소요 시간(초) : 39.008129358291626


In [3]:
# WILL_DELETE_COLUMNS_LIST=["LOG_SEQ", "SVR_REQ_TIME", "SVR_RESP_TIME", "X_DEVICE_ID", 
#                           "X_COUNTRY_CP", "X_DEVICE_MODEL", "X_DEVICE_FW_VERSION", 
#                           "USR_STAT", "DIALOG_ACT_TP", "DOMN_EXPERT_HIT", "SYS_ACT_TP",
#                           "TV_SRCH_QUERY", "HTTP_STAT_CODE", "ERROR_CODE", 
#                           "ERROR_MSG", "EXT_TIME", "EXT_NLU_RESP", "AUDIO_SRC", "NLU_ID",
#                           "OPT_QUERY", "X_DEVICE_LOGICAL_ID"]
WILL_DELETE_COLUMNS_LIST=["LOG_SEQ", "SVR_REQ_TIME", "SVR_RESP_TIME", "X_DEVICE_ID", 
                          "X_COUNTRY_CP", "X_DEVICE_MODEL", "X_DEVICE_FW_VERSION", 
                          "DIALOG_ACT_TP", "DOMN_EXPERT_HIT", "SYS_ACT_TP",
                          "TV_SRCH_QUERY", "HTTP_STAT_CODE", "ERROR_CODE", 
                          "ERROR_MSG", "EXT_TIME", "EXT_NLU_RESP", "NLU_ID",
                          "X_DEVICE_LOGICAL_ID",
                         "NBEST", "NBEST_1", "DVC_EPG", "SYS_UTTER", "DOMN_EXPERT", "MULTI_ACT", "ACT_CHOSEN", "ACT_CHANGED"]
FILTER_VALUE_X_DEVICE_COUNTRY = 'KR'

#발화 미 입력, 

In [4]:
#
# 데이터 전처리 부분
#
def preprocessing(df):
    start_time = time.time()
    #불필요한 column의 데이터 삭제 작업 수행
    try:
        df = df.drop(columns=WILL_DELETE_COLUMNS_LIST)
    except:
        print("except")

    #발화 칼럼에 데이터 없는 경우 해당 행 삭제
    df = df.dropna(subset=['USR_UTTER']) #USR_UTTER 있는 NaN row만 삭제. axis default는 row이기에,

    #원하는 국가로 데이터 필터링
    #Pandas Series 문자열 찾는 메서드 설명 잘되어있는 블로그
    #https://emilkwak.github.io/pandas-contains-multiple
    df = df[ df['X_DEVICE_COUNTRY'] == FILTER_VALUE_X_DEVICE_COUNTRY ]
    df = df[ df['USR_UTTER'] != "SHORT_KEY_NLP_HELP" ]
    #df = df[ (df['X_DEVICE_PLATFORM'].str.startswith('W18')) | (df['X_DEVICE_PLATFORM'].str.startswith('W19')) ]

    df.to_excel(FOLDER_PATH+"result_num"+".xlsx")
    print(df.index)
    print ("소요 시간(초) : " + str(time.time() - start_time))
    return df

In [6]:
globalDF = preprocessing(globalDF)

Int64Index([     12,      18,      78,     167,     184,     189,     198,
                206,     294,     299,
            ...
            1143076, 1143099, 1143107, 1143118, 1143140, 1143148, 1143242,
            1143254, 1143287, 1143288],
           dtype='int64', length=130738)
소요 시간(초) : 24.364927053451538


In [24]:
#MainAction 추출하는 funcition
#
# 제네릭하게 사용할 수 있는 방법 생각해 볼 것
#
def extract_mainAction(_df):
    #https://vincien.tistory.com/16
    #print (globalDF["MAIN_ACT_TP"].unique())
    extract_mainAction_count_Series = _df["MAIN_ACT_TP"].value_counts(normalize=False, sort=True, ascending=False,
                                               bins=None, dropna=True)
    extract_mainAction_percent_Series = _df["MAIN_ACT_TP"].value_counts(normalize=True, sort=True, ascending=False,
                                               bins=None, dropna=True)
    extract_mainAction_percent_Series = extract_mainAction_percent_Series * 100
    #df = pd.DataFrame([extract_mainAction_count_Series, extract_mainAction_percent_Series]) #, index=['a','b','c','d'], columns = ['kor', 'eng'])
    df = pd.concat([extract_mainAction_count_Series, extract_mainAction_percent_Series], axis=1) #merge를 하는데, 'dept' 칼럼을 기준으로 해라는 뜻
    df.index.name="mainAction"
    
    #후에 X-DEVICE_LANGUAGE랑 USER_TIME 따로 저장하여 추가할 것
    df['X-DEVICE_LANGUAGE'] = "ko-KR"
    print (df.shape)
    print (df.index)

    df.to_excel(FOLDER_PATH+"extracted_mainAction"+".xlsx")

In [25]:
extract_mainAction(DF1)

(142, 3)
Index(['tv_unknown_search', 'tv_open_search', 'tv_search_title', 'tv_shortcut',
       'google_assistant', 'tv_play', 'tv_play_title', 'uc_open_search_user',
       'web_unknown_search', 'tv_open_play',
       ...
       'tv_web_search_actor', 'tv_tv_off_setting_cancel',
       'tv_search_day_no_keyword', 'sh_device_temp_setting',
       'tv_control_not_available', 'sports_myteam_score',
       'tv_preference_channel_add', 'sh_device_status', 'tv_shortcut_livemenu',
       'tv_shortcut_energysaving'],
      dtype='object', name='mainAction', length=142)


In [7]:
def make_NE_tagging_corpous(df):
    ###df = pd.read_excel(FOLDER_PATH+"result_num"+".xlsx", encoding='UTF8')
    ###temp_dataframe = df[["MAIN_ACT_TP", "USR_UTTER", "NAMED_ENT"]]
    #temp_dataframe = globalDF[["MAIN_ACT_TP", "USR_UTTER", "NAMED_ENT"]]
    start_time = time.time()
    temp_dataframe = df
    temp_dataframe["ne"] = 1
    temp_dataframe["ne1"] = 1
    SEARCH_PATTERN1 = re.compile("\[(.+)\]", re.IGNORECASE)
    count = 0
    for idx, text in temp_dataframe.iterrows():
        #print(globalDF["USR_UTTER"][idx])
        try:
            #print(globalDF["USR_UTTER"][idx])
            USR_UTTER = df["USR_UTTER"][idx].lower()
            #print( type(USR_UTTER) )
            #USR_UTTER_SUB = re.sub('[\-]', '', USR_UTTER)
            USR_UTTER_SUB = USR_UTTER.replace('-', ' ')
            ##print(idx)
            if (count % 10000 == 0):
                print ('parent process : ' + str(os.getpid() ) + " df.index : " + str(df.index) + ", 처리 문장 갯수(1만개기준출력) : " + str(count) )
            
            original_text = USR_UTTER_SUB
            tag_original_text = USR_UTTER_SUB
            text = df["NAMED_ENT"][idx].lower()
            #print( type(text) )
            #원문에서 대괄호 제거 정규식( [] )
            text = ''.join(re.findall(SEARCH_PATTERN1, text))
            #NE가 2개 이상일 경우 ,로 split하여 List 생성
            text = text.split(",")
            #print("byb : " + " ".join(text))
            #',' 로 구분된 NE 리스트 반복문
            for i in text:
                #print(i)
                #NE명과 해당 NE Text 추출 하는 정규표현식
                temp1 = ''.join( re.findall("{\"(.+)\":\".+\"}", i) )
                temp2 = ''.join( re.findall("{\".+\":\"(.+)\"}", i) )
                temp1 = re.sub('[\"\']', '', temp1)
                temp2 = re.sub('[\"\']', '', temp2)
                #print(temp1)
                #print(temp2)

                temp2 = temp2.split(':')
                #print(temp2)
                #print("temp2[0] : " + temp2[0])
                if temp2[0] != "":
                    tag_original_text = tag_original_text.replace(temp2[0], "<" + temp2[0] + ":" + temp1 + ">")
                    original_text = original_text.replace(temp2[0], "<" + temp1 + ">")


                #print("original_text : " + original_text)
                #print("tag_original_text : " + tag_original_text)
                #print("----\n\n")
            count = count + 1
        except Exception  as e:
            original_text = temp_dataframe["USR_UTTER"][idx]
            tag_original_text = temp_dataframe["USR_UTTER"][idx]
            print("except")
            print(e)
        temp_dataframe["ne"][idx] = original_text
        temp_dataframe["ne1"][idx] = tag_original_text
     
    return temp_dataframe
    #temp_dataframe.to_excel(FOLDER_PATH+"make_NE_Corpus"+".xlsx")
    

In [8]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, numCores)
    pool = Pool(numCores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [9]:
start_time = time.time()
dataFrame = parallelize_dataframe( globalDF[["MAIN_ACT_TP", "USR_UTTER", "NAMED_ENT"]], make_NE_tagging_corpous )
dataFrame.to_excel(FOLDER_PATH+"make_NE_Corpus"+".xlsx")
print ("소요 시간(초) : " + str(time.time() - start_time))

parent process : 9314 df.index : Int64Index([    12,     18,     78,    167,    184,    189,    198,    206,
               294,    299,
            ...
            107638, 107639, 107642, 107643, 107647, 107650, 107652, 107658,
            107670, 107671],
           dtype='int64', length=8172), 처리 문장 갯수(1만개기준출력) : 0
parent process : 9315 df.index : Int64Index([107673, 107674, 107675, 107676, 107678, 107687, 107691, 107706,
            107707, 107710,
            ...
            147288, 147291, 147296, 147303, 147331, 147337, 147341, 147347,
            147349, 147355],
           dtype='int64', length=8172), 처리 문장 갯수(1만개기준출력) : 0
parent process : 9316 df.index : Int64Index([147356, 147373, 147378, 147386, 147387, 147397, 147399, 147404,
            147407, 147411,
            ...
            210125, 210126, 210142, 210143, 210145, 210158, 210160, 210162,
            210178, 210185],
           dtype='int64', length=8171), 처리 문장 갯수(1만개기준출력) : 0
parent process : 9317 df.index : Int64In

/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


parent process : 9322 df.index : Int64Index([566887, 566898, 566899, 566910, 566919, 566932, 566933, 566934,
            566946, 566947,
            ...
            627527, 627528, 627530, 627547, 627562, 627568, 627578, 627583,
            627617, 627621],
           dtype='int64', length=8171), 처리 문장 갯수(1만개기준출력) : 0


/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


parent process : 9323 df.index : Int64Index([627628, 627635, 627653, 627657, 627664, 627671, 627677, 627689,
            627690, 627702,
            ...
            685058, 685066, 685070, 685075, 685077, 685080, 685083, 685094,
            685099, 685103],
           dtype='int64', length=8171), 처리 문장 갯수(1만개기준출력) : 0


...

/home/yeongbin/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/yeongbin/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/


parent process : 9327 df.index : Int64Index([867623, 867625, 867635, 867641, 867646, 867652, 867654, 867659,
            867669, 867674,
            ...
            940007, 940012, 940023, 940033, 940036, 940040, 940048, 940052,
            940058, 940073],
           dtype='int64', length=8171), 처리 문장 갯수(1만개기준출력) : 0

/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.p

/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yeongbin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying

소요 시간(초) : 19.423267126083374


In [33]:
def sort_desc_mostSaidCorpus(_df):
    df = _df
    #df = tmp.drop(columns=['Unnamed: 0'])
    
    print(df.columns)
    corpus_dict={}
    corpus_list=[]
    for idx,s in df.iterrows():
        try:
            ne_corpus = df['MAIN_ACT_TP'][idx] + "-" + df['ne'][idx]
            if ne_corpus in corpus_dict:
                corpus_dict[ne_corpus] += 1
            else:
                corpus_dict[ne_corpus] = 1
        except:
            print("Except : can only concatenate str (not int) to str")
    #print(corpus_dict)
    
    for key in corpus_dict.keys():
        temp = key.split("-")
        temp.append(corpus_dict[key])
        corpus_list.append(temp)
    all_file_NE_comb_output = pd.DataFrame(corpus_list, columns=["MainAction","NE Combination(Output in order)","appear Count"])
    
    unique_mainAction_list = all_file_NE_comb_output['MainAction'].unique()
    outputFrame = pd.DataFrame(columns=("MainAction", "NE Combination(Output in order)", "appear Count"))
    num=0
    for mainAction in unique_mainAction_list:
        newFrame = []
        newFrame = all_file_NE_comb_output[all_file_NE_comb_output['MainAction'] == mainAction]
        newFrame = newFrame.sort_values(by=['appear Count'], ascending=[False])
        ###
        # 상위 95% 발화 패턴 추출하는 부분을 여기에 구현하여야 한다.
        totalAppearCount = newFrame['appear Count'].sum()
        endLoopCount = totalAppearCount * 0.9
        sumAppearCount = 0
        index = 0;
        while sumAppearCount < endLoopCount:
            sumAppearCount = sumAppearCount + newFrame['appear Count'].iloc[index]
            index = index + 1
            print ( "totalAppearCount : " + str(totalAppearCount) + ", sumAppearCount : " + str(sumAppearCount) + ", endLoopCount : " + str(endLoopCount))
        ###
        num = num + 1
        print(newFrame.columns)
        print(index)
        print(str(num) + ". " + mainAction)
        #현제까지 인덱스 슬라이싱이 안되는 이슈
        newFrame = newFrame.reset_index(drop=True)
        newFrame = newFrame[0:index]
        newFrame['rate'] = newFrame['appear Count']  / totalAppearCount * 100
        #df = df[ df['X_DEVICE_COUNTRY'] == FILTER_VALUE_X_DEVICE_COUNTRY ]
        #df = df[ df['USR_UTTER'] != "SHORT_KEY_NLP_HELP" ]
        
        print(newFrame)
        outputFrame = outputFrame.append(newFrame)
        print(outputFrame)
        
    
    #all_file_NE_comb_output = all_file_NE_comb_output.dropna(axis=0, ) # NaN row 삭제
    outputFrame.to_excel(FOLDER_PATH + "all_file_NE_comb_list" + ".xlsx")
    print("Process Finish!!!!")
    #output = pd.DataFrame(corpus_dict)
    #output = output.dropna(axis=0, ) # NaN row 삭제
    #print(output)
    #output.to_excel("D:/EIC/" + "test" + ".xlsx")
            


In [34]:
sort_desc_mostSaidCorpus(dataFrame)

Index(['MAIN_ACT_TP', 'USR_UTTER', 'NAMED_ENT', 'ne', 'ne1'], dtype='object')
totalAppearCount : 20692, sumAppearCount : 13293, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 15281, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 16796, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 17093, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 17376, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 17634, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 17784, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 17928, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 18039, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 18149, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 18253, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 18335, endLoopCount : 18622.8
totalAppearCount : 20692, sumAppearCount : 184

/home/yeongbin/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


totalAppearCount : 37630, sumAppearCount : 15678, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15680, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15682, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15684, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15686, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15688, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15690, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15692, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15694, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15696, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15698, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15700, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15702, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 15704, e

totalAppearCount : 37630, sumAppearCount : 20916, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20917, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20918, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20919, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20920, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20921, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20922, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20923, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20924, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20925, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20926, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20927, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20928, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 20929, e

totalAppearCount : 37630, sumAppearCount : 23439, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23440, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23441, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23442, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23443, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23444, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23445, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23446, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23447, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23448, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23449, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23450, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23451, endLoopCount : 33867.0
totalAppearCount : 37630, sumAppearCount : 23452, e

...

                    MainAction NE Combination(Output in order) appear Count  \
0              tv_search_title                     <ett_title>        13293   
1              tv_search_title                      <pr_actor>         1988   
2              tv_search_title                     <ett_genre>         1515   
3              tv_search_title          <pr_actor> <ett_title>          297   
4              tv_search_title         <ett_title> <ett_title>          283   
5              tv_search_title          <ett_title> <pr_actor>          258   
6              tv_search_title         <ett_title> <ett_genre>          150   
7              tv_search_title                           hello          144   
8              tv_search_title         <ett_genre> <ett_title>          111   
9              tv_search_title                   <ett_title> 2          110   
10             tv_search_title          <ett_title> <qt_order>          104   
11             tv_search_title                <bs_br

                    MainAction NE Combination(Output in order) appear Count  \
0              tv_search_title                     <ett_title>        13293   
1              tv_search_title                      <pr_actor>         1988   
2              tv_search_title                     <ett_genre>         1515   
3              tv_search_title          <pr_actor> <ett_title>          297   
4              tv_search_title         <ett_title> <ett_title>          283   
5              tv_search_title          <ett_title> <pr_actor>          258   
6              tv_search_title         <ett_title> <ett_genre>          150   
7              tv_search_title                           hello          144   
8              tv_search_title         <ett_genre> <ett_title>          111   
9              tv_search_title                   <ett_title> 2          110   
10             tv_search_title          <ett_title> <qt_order>          104   
11             tv_search_title                <bs_br

                    MainAction NE Combination(Output in order) appear Count  \
0              tv_search_title                     <ett_title>        13293   
1              tv_search_title                      <pr_actor>         1988   
2              tv_search_title                     <ett_genre>         1515   
3              tv_search_title          <pr_actor> <ett_title>          297   
4              tv_search_title         <ett_title> <ett_title>          283   
5              tv_search_title          <ett_title> <pr_actor>          258   
6              tv_search_title         <ett_title> <ett_genre>          150   
7              tv_search_title                           hello          144   
8              tv_search_title         <ett_genre> <ett_title>          111   
9              tv_search_title                   <ett_title> 2          110   
10             tv_search_title          <ett_title> <qt_order>          104   
11             tv_search_title                <bs_br

                    MainAction NE Combination(Output in order) appear Count  \
0              tv_search_title                     <ett_title>        13293   
1              tv_search_title                      <pr_actor>         1988   
2              tv_search_title                     <ett_genre>         1515   
3              tv_search_title          <pr_actor> <ett_title>          297   
4              tv_search_title         <ett_title> <ett_title>          283   
5              tv_search_title          <ett_title> <pr_actor>          258   
6              tv_search_title         <ett_title> <ett_genre>          150   
7              tv_search_title                           hello          144   
8              tv_search_title         <ett_genre> <ett_title>          111   
9              tv_search_title                   <ett_title> 2          110   
10             tv_search_title          <ett_title> <qt_order>          104   
11             tv_search_title                <bs_br

                    MainAction NE Combination(Output in order) appear Count  \
0              tv_search_title                     <ett_title>        13293   
1              tv_search_title                      <pr_actor>         1988   
2              tv_search_title                     <ett_genre>         1515   
3              tv_search_title          <pr_actor> <ett_title>          297   
4              tv_search_title         <ett_title> <ett_title>          283   
5              tv_search_title          <ett_title> <pr_actor>          258   
6              tv_search_title         <ett_title> <ett_genre>          150   
7              tv_search_title                           hello          144   
8              tv_search_title         <ett_genre> <ett_title>          111   
9              tv_search_title                   <ett_title> 2          110   
10             tv_search_title          <ett_title> <qt_order>          104   
11             tv_search_title                <bs_br